# PyAdventure
This Python program is a choose-your-own-adventure text game! The player can type what action they would like to do. The program has a list of keywords it is looking for in an user's input. Each keyword corresponds to an action. When it finds a keyword, it will perform the action associated with the word. If the user misspells a word, the spellchecker will fix their mistake.
For the spellchecker, it reads in a file with a wordlist generated from the Brown corpus. This is to save time from computing it each time the game is run.

In [8]:
# import re
from collections import Counter
from nltk.corpus import brown
import random

class Player():
    """ class pertaining to the player's movement and interactions"""
    inventory = []
    def __init__(self, name = "Nameless Adventurer", health = 15, coordinates = (0,0)):
        self.name = name #may be good to get player's input for name in beginning
        self.health = health
        self.coordinates = (0, 0)

        #methods to move in all four cardinal directions
    def moveNorth(self):
        """Updates the current coordinates of the player and moves them north."""
        currentCoord = self.coordinates #get current coordinates
        editCoord = [currentCoord[0], currentCoord[1]] #transform to list to edit
        editCoord[1] += 1
        self.coordinates = tuple(editCoord) #set updated coordinates

    def moveEast(self):
        """Updates the current coordinates of the player and moves them east."""
        currentCoord = self.coordinates
        editCoord = [currentCoord[0], currentCoord[1]]
        editCoord[0] += 1
        self.coordinates = tuple(editCoord)

    def moveSouth(self):
        """Updates the current coordinates of the player and moves them south."""
        currentCoord = self.coordinates
        editCoord = [currentCoord[0], currentCoord[1]]
        editCoord[1] -= 1
        self.coordinates = tuple(editCoord)

    def moveWest(self):
        """Updates the current coordinates of the player and moves them west."""
        currentCoord = self.coordinates
        editCoord = [currentCoord[0], currentCoord[1]]
        editCoord[0] -= 1
        self.coordinates = tuple(editCoord)

    def movement(self, keywordList):
        """Find's the direction keyword placed in the keywordList[1] ([action, direction]) to find the direction the player wants"""
        if len(keywordList) != 2: #The cases where an item was found along with direciton, (item, direction), therefore no movement. ex: "key go north"
            print(f"I don't understand what you want to do with that {keywordList[0]}.")
            return

        if keywordList[1] == 'north' or keywordList[1] == 'up':
            print("You ventured north")
            self.moveNorth()
        elif keywordList[1] == 'east' or keywordList[1] == 'right':
            self.moveEast()
            print("You ventured east")
        elif keywordList[1] == 'south' or keywordList[1] == 'down':
            self.moveSouth()
            print("You ventured south")
        elif keywordList[1] == 'west' or keywordList[1] == 'left':
            self.moveWest()
            print("You ventured west")
        else:
            print("You want to go where?")
            return

    # method to add item to a player's inventory
    def addItem(self, item):
        """Adds an item to the player's inventory."""
        inventory = self.inventory
        inventory.append(item)

    # method to update health by adding a specified number of HP
    def updateAddHealth(self, number):
        """Updates the health of the player by adding a specified number of health points."""
        health = self.health
        health = health + number

    # method to update healthy by subtracting a specified number of HP
    def updateSubHealth(self, number):
        """Updates the health of the player by subtracting a specified number of health points."""
        health = self.health
        health = health - number

    #extracts keywords from player input and current map's items list
    def findKeyword(self, itemsDict, userInput):
        """Finds keywords in the user's input and finds the appropriate action that corresponds to that action.

        Arguments
        ----------
        itemsDict: dictionary
            This dictionary contains interactable items of the current coordinates location

        userInput: string
            Takes the user's input to tokenize and is then searched for items > actions > direction

        returns back to main() to get next user input
        
        Examples
        --------
        "I would like to go to the west side down the hall" as a user input
            first, search for a room keywords, none found
            next, search for action, ['go'] is found
            finally, search for direction, ['go', 'west'] ([ACTION, DIRECTION]) since action is a movement, movement() method will be used
        
        "pick up the key on the table"
            room keyword ['key'] found
            action found['key', 'pick']
            no direction searched for, ([ITEM, ACTION]) will go through the '#select action' area to figure out what to do with the item
        
        "walk noirth"
            room keyword, nothing
            action found ['walk']
            no direction found, spellchecker() used which will return ['walk', 'north'] back to this method
            
        """
        item = [] #found keyword items
        action = ['move', 'go', 'walk', 'take', 'open', 'bag', 'inventory', 'inv', 'unlock', 'grab', 'pick', 'get', 'look', 'examine', 'use', 'attack'] #action keywords
        direction = ['north', 'east', 'south', 'west', 'up', 'down', 'left', 'right'] #direction keywords
        user = tokenizeInput(userInput) #tokenize input

        if 'quit' in user or 'end' in user or 'dc' in user: #pull up quit menu
            quitMenu()

        for word in user: #search for item in tokenized input
            if word in itemsDict:
                item.append(word)
        for word in user:
            if word in action: #search for action
                item.append(word)
        if item != 2:
            for word in user:
                if word in direction: #search for directional word
                    item.append(word)

        #No keywords Found
        if len(item) < 2:
            item = spellChecker(user, action, direction)
            if len(item) != 2: #unable to find what user meant to say
                print("I don't understand what you want to do.")
                return

        #select a direction to move
        if 'move' in item or 'go' in item or 'walk' in item:
            self.movement(item)
            return

        #opening inventory
        if 'bag' in item or 'inventory' in item or 'inv' in item:
            self.actionBag()
            return

        #USE ITEMS (should make new list)
        if 'use' in user: #user wants to use
            self.actionUse(user)
            return

        # attack in room
        if 'attack' in user: # user wants to attack
            self.actionAttack()
            return

        #SELECT ACTIONS
        if itemsDict.get(item[0]): #check if item exists on the map
            itemsDict = itemsDict.get(item[0]) #set dictionary to current item's available actions
            if len(item) >= 2:
                if item[1] in itemsDict: #traversed dictionary has different options
                    #OPEN / UNLOCK
                    if item[1] == 'open' or item[1] == 'unlock':
                        self.actionOpen(item)
                        return

                    #Looting items
                    if 'pick' in item: #'pick up' is special case b/c of 2 word action
                        print(f"You {item[1]} up {item[0]}.")
                    else:
                        print("You", item[1], item[0]) #normal pick up for items
                    self.actionLoot(item)
                    return
                #LOOK / EXAMINE (traversed dictionary is a string)
                if item[1] == 'look' or item[1] == 'examine':
                    self.actionExamine(item)
                    return
                else:
                    print(f"You want to do what with the {item[0]}?") #action is unavailable for item
                    return
            else:
                print(f"You want to do what with the {item[0]}?") #action is unavailable for item
                return

        #when only a direction is found
        for word in direction:
            if word in item:
                if item[0] == 'pick':
                    print(f"I don't know what you want to {item[0]} up.")
                    return
                if item[0] == 'look':
                    print(f"Look at what?")
                    return
                if item[0] == 'examine':
                    print(f"Examine what?")
                    return
                print(f"I don't understand what you mean.")
                return

        #when only action is found
        for word in action:
            if word in item:
                if item[0] == 'pick':
                    print(f"I don't know what you want to {item[0]} up.")
                    return
                elif item[0] == 'look':
                    print(f"I don't know what you want to {item[0]} at.")
                    return
                print(f"I don't know what you want to {item[0]}.")
                return

        print("you missed a filter") #Should never reach here
        return

    def actionOpen(self, item):
        """Opens the door to a room by adding the value 'False' (door is not locked) to the openable item's list of interactions;
            When the item is unlocked, room description is changed by adding the 'True' value to the list of descriptions in the coordinate's current location
        """
        #need to change this to instead change the value True to False instead of just adding False to the list of interactinos for the door
        #need to add windows mayb / definitely chests eventually
        if item[0] == 'door':#door interactions
            if True in room.get(self.coordinates).get('items').get('door'): #see if it's locked
                if 'key' in self.inventory: #if you have a key in your inventory
                    print("You unlocked the door")
                    self.inventory.remove('key') #use key and remove it from inventory
                    room.get(self.coordinates).get('items').get('door').append(False) #add False meaning door is unlocked
                    room.get(self.coordinates).get('description').append(True) #change to 2nd description for editable rooms
                    return
                else:
                    print("You don't have the key.") #you dont have a key in your inventory
                    return
        else:
            print("You can't open it.")
     
    def actionLoot(self, item):
        """Adds an item to the player's inventory;
           This method also changes the room description by adding 'True' to the description list when an item is removed from the location
        """
        self.addItem(item[0]) #put item in inventory
        room.get(self.coordinates).get('items').pop(item[0]) #remove item from map
        room.get(self.coordinates).get('description').append(True) #change to 2nd description
        return
    
    def actionUse(self, user):
        """Action that makes player use an item from their inventory."""
        #keys should also be reworked to deal with multiple items in the rooms. A key shouldn't open everything
        #Make colored keys maybe? 'blue key', 'red key' etc
        if 'key' in user: #use keys
            if 'key' in self.inventory: #key is in inventory
                ###Using key on doors###
                #This needs to be reworked to deal with rooms having multiple doors, maybe if there's more than 1 door player must specify 'wooden', 'steel' door etc
                if room.get(self.coordinates).get('items').get('door'): #if there's a door in room
                    if True in room.get(self.coordinates).get('items').get('door'): #and door is locked
                        print("You used the key")
                        self.inventory.remove('key') #use key and remove it from inventory
                        room.get(self.coordinates).get('items').get('door').append(False) #add False meaning door is unlocked
                        room.get(self.coordinates).get('description').append(True) #change to 2nd description for editable rooms
                        return
                    else:
                        print("The key doesn't work anywhere here.")
                        return
                else:
                    print("You don't have a key in your bag.")
                    return

        elif 'potion' in user: #use potions
            if 'potion' in self.inventory:
                print("future potion stuff.")
                # adds one health point to player's health
                # can add functionality to add more complex potions
                self.updateAddHealth(1)
                # what else should potions be able to do?
                    #if it's a normal health potion, just healing is fine. 
                    #we need to make sure when the potion is used it's removed from the inventory
                    #we also need to figure how to deal with having multiples of the same item in inventory
                return
            else:
                print("You're out of potions.")
                return
        else:
            print("You can't use that.")

    def actionExamine(self, item):
        """Action user can take to examine something in the room."""
        if item[0] == 'window': #WINDOW interactions
            print(room.get(self.coordinates).get('items').get('window')[0])
            return
        elif item[0] == 'door': #DOOR interactions
            print("It's a normal door")
            return
        elif item[0] == 'key':
            print("It's boring silver key")
            return
        print("Nothing special here.")
        return

    def actionBag(self):
        """Action player can take to look at their inventory."""
        print("You look in your bag")
        if self.inventory == []:
            print("You have nothing")
        else:
            i = 0
            for item in self.inventory:
                print(f"-{self.inventory[i]}")
        return
    
    def actionAttack(self):
        """Action taken when player is under attack."""
        print("You are under attack")
        # choose a random number between 0 and n - depends how sophisticated rng should be
            #i was thinking having different weapons with different ranges, like you can start with a dagger in your inventory which has a chance of hitting 1~3. depending on the monster, chance to hit changes?
        # maybe 0 can be like player dodged the attack?
        attackNum = random.randint(0, 1)
        #self.updateSubHealth(attackNum) # right now, subtracts number of HP by rng
            #this could be solved with monster attack ranges just like weapons. ex: rat does 1~2 damage or something with a chance of missing and a dog does 2~4 w/e
        if "dagger" in self.inventory: # trying out a weapon
            print("You used a dagger to fight the attack and are not harmed")
            self.updateSubHealth(0) # player defends themselves against the attack
        else:
            print(f"You were attacked and lost {attackNum} health points")
            self.updateSubHealth(attackNum) # if they don't defend themselves then player will get attacked random HP
############################################################################################################
#class Location:
#    def __init__(self, items = {}, description = "", coordinates = (0,0)):
#        self.items = items #list of items in a room
#        self.description = description
#        self.coordinates = coordinates # which set of coordinates it is on the map
############################################################################################################

###MENUS###

#ask user if they want to quit
# whenever user types quit or end, systemExit exception ends game
def quitMenu():
    """Simple loop asking user if they would like to quit"""
    yes = ['yes', 'ye', 'y', 'ya', 'yer']
    no = ['no', 'n']
    print("Quit?")
    while True:
        answer = input()
        answer = cleanUp(answer)
        if answer in yes:
            import sys
            # Commented error
            sys.exit("##########\nThanks for playing!\n######################")
        elif answer in no:
            return False

#############################################################################################################

###STRING CLEANING###

#tokenize strings after cleaning them
def tokenizeInput(userInput):
    """Cleans up the user's input and finds all words."""
    cleanUp(userInput)
    tokenized = re.findall(r"\w+", userInput.lower())
    return tokenized #returns user input tokenized

#removes unwanted characters / spaces from strings
def cleanUp(userInput):
    """Cleans up the user's input by deleting any punctuation and removing whitespace and making it one space."""
    userInput = re.sub(r"[\.\?!,;-]", r"", userInput) #clean string
    userInput = re.sub(r"\s+", r" ", userInput) #remove extra white space
    return userInput.lower() #returns cleaned lower final string

##############################################################################################################

###SPELL CHECK METHODS###

def createWordlist(corpus = brown.words(), limit = 1000):
    """Creates a corpus to refer to for misspelled words"""
    CorpusSet = set(corpus[:limit]) if limit else set(corpus)
    wordList = []
    for word in CorpusSet:
        wordList.append(word)
    return wordList #returns a list of 'limit' amount of words

def charNgrams(wordlist, n = 2, lowercase = True):
    """extracts character ngrams for basic spell checking from the given wordlist
    
    Arguments
    ----------
    wordlist: list
        Wordlist with words that are correctly spelled
        
    n: integer
        Default set at 2, number for n-grams
        
    lowercase: boolean
        Default set to True, truth value for whether the letters of words are lowercase
    """
    ngramList = []
    if lowercase: #set uppercase to lowercase
        lowerList = []
        i = 0
        for word in wordlist:
            lowerList.append(wordlist[i].lower())#add lowercase words to new list
            i += 1
        wordlist = lowerList #set wordlist to new list

    for word in wordlist: #create list of ngrams
        i = 0
        for letter in word:
            ngramList.append(word[i:i+n])
            i += 1

    return Counter(ngramList)

def isMisspelled(word, wordlist, ngrams={}):
    """method used for checking if the word in question exists in the corpus wordlist or if its character ngrams are valid;
       if the word char ngrams do not exist in the dictionary of corpus char ngrams, probably misspelled
       if misspelled return True
    """
    #word is in list, not misspelled
    if word in wordlist:
        return False
    #extract ngrams of the word
    checkWord = [word]
    checkWordGrams = charNgrams(checkWord)
    #place ngrams in list From counter
    wordNgrams = list(checkWordGrams.keys())
    #loop through every ngram in word
    i = 0
    for ngram in wordNgrams:
        #if ngram not in ngrams dictionary, misspelled
        if wordNgrams[i] not in ngrams:
            return True
        else:
            i += 1
    return False #word is probably not misspelled

def constructGrid(badWord, keyWord):
    """Compute the default cost for each edge."""
    # grid is now a dictionary instead of a list
    grid = {(x,y): {}
            for x in range(len(badWord) + 1)
            for y in range(len(keyWord) + 1)}
    for x, y in grid:
        # add deletion edge if there is a node to the left
        if x > 0:
            grid[(x,y)][(x-1, y)] = 1
        # add insertion edge is there is a node above
        if y > 0:
            grid[(x,y)][(x, y-1)] = 1
        # add substitution edge; check if cost is 0
        if x > 0 and y > 0:
            grid[(x,y)][(x-1, y-1)] =\
                0 if badWord[x-1] == keyWord[y-1] else 1
    return grid

def cost(node, grid):
    """Calculate the cost of the optimal path to node through the grid."""
    if node == (0,0):
        return 0
    else:
        lowest = min([cost(neighbor, grid) + edgeCost
                      for neighbor, edgeCost in grid[node].items()])
        return lowest

def levenshteinDistance(badWord, keyWord):
    """Calculate Levenshtein distance between the mispelled word and keyword."""
    return cost((len(badWord), len(keyWord)), constructGrid(badWord, keyWord))

def spellChecker(userInput, actionList, directionList):
    """Uses levenshtein distance on found misspelled words with the action/direction words
       It will return a list in the format of [action, direction];
       spell checking for items isn't done to avoid finding an item by accident

    Arguments
    ----------
    userInput: list
        Takes the tokenized input to try to find misspelled key words

    actionList: list
        The main list of actions a user to able to take in the game from the findKeyword() method

    directionList: list
        The main list of directions a user is able to move in the game from the findKeyword() method

    """
    #corpus = createWordlist() #wordList of corpus     #should be saved somewhere so it isn't recalled everytime slowing it down in actual game
    # we should be able to save it in a file somewhere and then load the file
    #f = open("wordlist.txt", "w+")
    #for word in corpus:
    #    f.write(word)
    #    f.write("\n")
    #f.close()
    #print(corpus)
    corpus = []
    f1 = open("wordlist.txt", "r") # open created text file
    f2 = f1.readlines()
    for x in f2: # read each word and append it into the corpus
        corpus.append(x)
    f1.close()
    corpusNgrams = charNgrams(corpus) #char ngrams of wordList
    misspelled = []
    spellChecked = []
    keywordList = actionList + directionList #used to find all keywords

    for word in userInput: #extract correctly spelled keywords
        if word in keywordList:
            spellChecked.append(word)
            userInput.remove(word) #remove the word from the list so no duplicates are found

    for word in userInput: #extract misspelled words
        if isMisspelled(word, corpus, corpusNgrams):
            misspelled.append(word) #add to misspelled word list

    #find an action misspelled
    for direction in directionList:
        if direction in spellChecked: #if we already have a direction, need an action
            for word in misspelled: #for every found misspelled word
                for action in actionList:
                    if (levenshteinDistance(word, action) < 3): #fixed keyword list should have [action, direction]
                        spellChecked.append(action)
                        quickSwap = [spellChecked[1], spellChecked[0]] #change order to [action, direction]
                        return quickSwap

    #direction misspelled
    for action in actionList:
        if action in spellChecked: #if we already have an action, should be directions only
            for word in misspelled: #for every found misspelled word
                for direction in directionList:
                        if (levenshteinDistance(word, direction) < 3):
                            spellChecked.append(direction)
                            return spellChecked
                        
    #everything misspelled
    for word in misspelled:
        for keyword in keywordList:
            if (levenshteinDistance(word, keyword) < 2): #lowered distance to 2 so find at least 1 very similar key word
                spellChecked.append(keyword)
                if len(spellChecked) == 2: #if you already found [action, direction]
                    return spellChecked
                if spellChecked[0] in actionList:
                    #search for direction
                    for direction in directionList:
                        if (levenshteinDistance(word, direction) < 3):
                            spellChecked.append(direction)
                else:
                    #search for action
                    for action in actionList:
                        if (levenshteinDistance(word, action) < 3): #fixed keyword list should have [action, direction]
                            spellChecked.append(action)
                            quickSwap = [spellChecked[1], spellChecked[0]] #change order to [action, direction]

    return spellChecked

#####################################################################################################################

def play():
    """Main play function that let the user play the game."""
    print("Welcome to the text adventure game!")
    print("What is your player's name?")
    name = input()
    player = Player(name) # assigns inputted name as player
    print(f"Welcome {name}!\nYou begin your adventure in the center of test room facility.")
    print("Please type the action you would like to do...\n")
    endgame = False
    while not endgame:
        currentPos = player.coordinates #get player's current position
        description = room.get(currentPos).get('description') #current positions description
        if True not in description:
            print(description[0]) #normal description of room
        else:
            print(description[1]) #description of room if cleared
        roomItems = room.get(currentPos).get('items') #get dictionary of current room's items
        while True:
            userInput = input()
            player.findKeyword(roomItems, userInput) #find keywords like item/action to do something
            requestedMove = player.coordinates #where player wants to move
            if requestedMove not in room: #if the player moves to coordinate that doesn't exist
                print("you walked into a wall...")
                player.coordinates = currentPos
                print("\n***********************************************\n")
                break
            #check if door has a lock
            if room.get(currentPos).get('items').get('door'):
                #True = locked, False = unlocked
                if False not in room.get(currentPos).get('items').get('door') and room.get(requestedMove).get('items').get('door'):
                    #if the door is locked in current room AND there the door requested room is connected with the door
                    player.coordinates = currentPos #move player back into last position
                    print("You can't go this way,\nThe door is locked.")
                    print("\n***********************************************\n")
                    break
                else:
                    print("\n***********************************************\n")
                    break
            else:
                print("\n***********************************************\n")
                break


###TESTING MAP###
room = {(0,0):{'items':{},
               'description':["[Center Room]\nA boring test room with doors in all 4 directions"]},

       (1,0):{'items':{
                    'door':['open', 'unlock', True]},
              'description':['[East Room]\nYou entered a room with a locked door on the east wall.', 'East room\nRoom with an opened door on the east side']},

       (0,1):{'items':{
                    'key':['get','take', 'grab', 'pick']},
              'description':["[North Room]\nAnother empty-.. oh wow! Look a key on the floor", "North Room\nNormal test room with now no key on the floor\nPsst, check out your inventory by typing 'bag' or 'inventory'"]},

       (-1,0):{'items':{
                    'window':["You peek out the window and see absolutely nothing..\nWhat did you expect?"]},
               'description':["[West Room]\nYou entered an empty room with a single window on the west side.\n'Look'/'Examine' test room"]},

       (0,-1):{'items':{},
              'description':["[South Room]\nYou entered a really boring empty test room\nCombat will be tested here eventually~"]},

       (2,0):{'items':{
                    'door':['open', False]},
             'description': ["You're a winner, yaaaay.\nYou may quit out of the game now."]}}

###################################################################################################################

#make a story
#make combat system, use rng to attack, can use item in inventory with attacks "attack with dagger", each weapon = integer for dmg range
#deal with multiple doors in room (wooden doors, steel door maybe)
#sentence parsing
#deal with multiple keywords in a sentence (currently picks out first keyword that pops up)
#searching rooms

play()

Welcome to the text adventure game!
What is your player's name?


 ry

Welcome ry!
You begin your adventure in the center of test room facility.
Please type the action you would like to do...

[Center Room]
A boring test room with doors in all 4 directions


 look at

Look at what?

***********************************************

[Center Room]
A boring test room with doors in all 4 directions


...

I don't understand what you want to do.

***********************************************

[Center Room]
A boring test room with doors in all 4 directions


Quit?


 y

SystemExit: ##########
Thanks for playing!
######################

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
